In [ ]:
# compile/install code that is needed for this demo
# if it says that you need to restart the runtime, go ahead and do that,
# then run this step again to make sure the installs are complete

!pip install git+https://github.com/nghorbani/human_body_prior
!pip install omegaconf
!pip install loguru
!pip install trimesh

In [ ]:
# connect to your google drive
# alternatively, upload the VPoserTest directory/files into Colab

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#filenames for loading VPoser VAE network, neutral SMPL body model, AMASS sample data

from os import path as osp

support_dir = '/content/gdrive/MyDrive/VPoserModelFiles/'

expr_dir = osp.join(support_dir,'vposer_v2_05/') #'TRAINED_MODEL_DIRECTORY'
bm_fname =  osp.join(support_dir,'smplx_neutral_model.npz')    #'PATH_TO_SMPLX_model.npz'  neutral smpl body model
sample_amass_fname = osp.join(support_dir, 'amass_sample.npz')  # a sample npz file from AMASS


print(expr_dir)
print(bm_fname)
print(sample_amass_fname)

In [ ]:
#Loading SMPLx Body Model
import torch
import numpy as np

# Choose the device to run the body model on, cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device is', device)

from human_body_prior.body_model.body_model import BodyModel
bm = BodyModel(bm_fname=bm_fname).to(device)


In [ ]:
#Loading VPoser VAE Body Pose Prior
from human_body_prior.tools.model_loader import load_model
from human_body_prior.models.vposer_model import VPoser

vp, ps = load_model(expr_dir, model_code=VPoser,
                              remove_words_in_model_weights='vp_model.',
                              disable_grad=True,
                              comp_device=device)
vp = vp.to(device)

In [ ]:
# Prepare the body poses from amass sample file
#  indexing [3:66] removes global rotation, hands/fingers, and anything else other than 21 major body joints
amass_body_pose = np.load(sample_amass_fname)['poses'][:, 3:66]
amass_body_pose = torch.from_numpy(amass_body_pose).type(torch.float).to(device)
print('amass_body_pose.shape', amass_body_pose.shape)

In [ ]:
# run the encoder on all frames
amass_body_poZ = vp.encode(amass_body_pose).mean
print('amass_body_poZ.shape', amass_body_poZ.shape)

In [ ]:
# run the decoder on all frames
amass_body_pose_rec = vp.decode(amass_body_poZ)['pose_body'].contiguous().view(-1, 63)
print('amass_body_pose_rec.shape', amass_body_pose_rec.shape)

In [ ]:
#get vertices and faces of a polygonal mesh model for each body pose

from human_body_prior.tools.omni_tools import copy2cpu as c2c
import trimesh

originalPoses = {'pose_body':amass_body_pose}
recoveredPoses = {'pose_body':amass_body_pose_rec}

bmodelorig = bm(**originalPoses);
bmodelreco = bm(**recoveredPoses);
vorig = c2c(bmodelorig.v)
vreco = c2c(bmodelreco.v)
faces = c2c(bm.f)

T, num_verts = vorig.shape[:-1]



In [ ]:
#visualize one frame's body pose before (grey) and after (purple) encode-decode
fIdx = 140
verts = vorig[fIdx]
mesh1 = trimesh.base.Trimesh(verts, faces)
mesh1.visual.vertex_colors = [254, 254, 254]
verts = vreco[fIdx]
mesh2 = trimesh.base.Trimesh(verts, faces)
mesh2.visual.vertex_colors = [254, 66, 200]
mesh2.apply_translation([1, 0, 0])  #use [0,0,0] to overlay them on each other
meshes = [mesh1, mesh2]
trimesh.Scene(meshes).show()

In [ ]:
#visualize a temporal subsequence of poses spatially (use mouse to rotate view)
#note that encoding followed by decoding is not a lossless process,
#it can introduce a certain amount of error all by itself

meshes = []
for fIdx in range(0,200,10):
    verts = vorig[fIdx]
    mesh1 = trimesh.base.Trimesh(verts, faces)
    mesh1.visual.vertex_colors = [254, 254, 254]
    mesh1.apply_translation([0, 0, fIdx*.07])
    meshes.append(mesh1)
    verts = vreco[fIdx]
    mesh1 = trimesh.base.Trimesh(verts, faces)
    mesh1.visual.vertex_colors = [254, 150, 200]
    mesh1.apply_translation([0, 0, fIdx*.07])
    meshes.append(mesh1)

trimesh.Scene(meshes).show()

In [ ]:
# extract and visualize 23 body joints before and after encode-decode process
# for a pose where error between original pose and decoded pose is rather large.
# why 23 instead of 21 mentioned earlier?  There are two extra joints somewhere
# that are not among the 21 rotatable body joints used by VPoser.

fIdx = 130

verts = vorig[fIdx]
mesh1 = trimesh.base.Trimesh(verts, faces)
mesh1.visual.vertex_colors = [254, 254, 254]
verts = vreco[fIdx]
mesh2 = trimesh.base.Trimesh(verts, faces)
mesh2.visual.vertex_colors = [254, 66, 200]
mesh2.apply_translation([0, 0, 0])  #use [0,0,0] to overlay them on each other
meshes = [mesh1, mesh2]


#get the 23 major 3D body joints
joints = c2c(bmodelorig.Jtr[fIdx])
origjoints = joints[0:23, :]   #ignore finger joints
joints = c2c(bmodelreco.Jtr[fIdx])
recojoints = joints[0:23, :]   #ignore finger joints

print(origjoints.shape, recojoints.shape)
for i in range(origjoints.shape[0]):
    sphere = trimesh.primitives.Sphere(radius=.02, center=origjoints[i,:])
    sphere.apply_translation([1, 0, 0])
    sphere.visual.vertex_colors = [254, 254, 254]
    meshes.append(sphere)
    sphere = trimesh.primitives.Sphere(radius=.02, center=recojoints[i,:])
    sphere.apply_translation([1, 0, 0])
    sphere.visual.vertex_colors = [254, 150, 200]
    meshes.append(sphere)

trimesh.Scene(meshes).show()

